In [0]:
#@title Copyright 2020 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_REST_API_computation.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/Earth_Engine_REST_API_computation.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Computations with the Earth Engine REST API

***Note:*** *The REST API contains new and advanced features that may not be suitable for all users.  If you are new to Earth Engine, please get started with the [JavaScript guide](https://developers.google.com/earth-engine/getstarted).*

The [Earth Engine REST API quickstart](quickstart) shows how to access blocks of pixels from an Earth Engine asset.  Suppose you want to apply a computation to the pixels before obtaining the result.  This guide shows how to protoype a computation with one of the client libraries, serialize the computation graph and use the REST API to obtain the computed result.  This corresponds to a `POST` request to one of the compute endpoints, for example [`computePixels`](https://developers.google.com/earth-engine/reference/rest/v1alpha/projects.image/computePixels), [`computeFeatures`](https://developers.google.com/earth-engine/reference/rest/v1alpha/projects.table/computeFeatures) or the generic [`value.compute`](https://developers.google.com/earth-engine/reference/rest/v1alpha/projects.value/compute).  Specifically, this example demonstrates getting the mean of pixels in each image of an `ImageCollection` in each feature of a `FeatureCollection`.

## Before you begin

Follow [these instructions](https://developers.google.com/earth-engine/earthengine_cloud_project_setup#apply-to-use-earth-engine) to:

1. Apply for Earth Engine
2. Create a Google Cloud project
3. Enable the Earth Engine API on the project
4. Create a service account

**Note**: To complete this tutorial, you will need a service account that is registered for Earth Engine access.  See [these instructions](https://developers.google.com/earth-engine/service_account#register-the-service-account-to-use-earth-engine) to register a service account before proceeeding.  

## Authenticate to Google Cloud

The first thing to do is login so that you can make authenticated requests to Google Cloud.  You will set the project at the same time.  Follow the instructions in the output to complete the sign in.

In [0]:
# INSERT YOUR PROJECT HERE
PROJECT = 'your-project'
!gcloud auth login --project {PROJECT}

## Obtain a private key file for your service account

You should already have a service account registered to use Earth Engine.  If you don't follow [these instructions](https://developers.google.com/earth-engine/service_account#create-a-service-account) to get one.  Copy the email address of your service account into the following cell.  (The service account must already be registered to use Earth Engine).  In the following, the `gsutil` command line is used to generate a key file for the service account.  They key file will be created on the notebook VM.

In [0]:
# INSERT YOUR SERVICE ACCOUNT HERE
SERVICE_ACCOUNT='your-service-account@your-project.iam.gserviceaccount.com'
KEY = 'key.json'

!gcloud iam service-accounts keys create {KEY} --iam-account {SERVICE_ACCOUNT}

## Start an `AuthorizedSession` and test your credentials

Test the private key by using it to get credentials.  Use the credentials to create an authorized session to make HTTP requests.  Make a `GET` request through the session to check that the credentials work.

In [0]:
from google.auth.transport.requests import AuthorizedSession
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(KEY)
scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform'])

session = AuthorizedSession(scoped_credentials)

url = 'https://earthengine.googleapis.com/v1alpha/projects/earthengine-public/assets/LANDSAT'

response = session.get(url)

from pprint import pprint
import json
pprint(json.loads(response.content))


## Serialize a computation

Before you can send a request to compute something, the computation needs to be put into the Earth Engine expression graph format.  The following demonstrates how to obtain the expression graph.

### Authenticate to Earth Engine

Get Earth Engine scoped credentials from the service account.  Use them to initialize Earth Engine.

In [0]:
import ee

# Get some new credentials since the other ones are cloud scope.
ee_creds = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY)
ee.Initialize(ee_creds)


### Define a computation

Prototype a simple computation with the client API.  Note that the result of the computation is a `FeatureCollection`.
To check that the computation can succeed without errors, get a value from the first `Feature` (the mean NDVI in the polygon).

In [0]:
# A collection of polygons.
states = ee.FeatureCollection('TIGER/2018/States')
maine = states.filter(ee.Filter.eq('NAME', 'Maine'))

# Imagery: NDVI vegetation index from MODIS.
band = 'NDVI'
images = ee.ImageCollection('MODIS/006/MOD13Q1').select(band)
image = images.first()

computation = image.reduceRegions(
  collection=maine, 
  reducer=ee.Reducer.mean().setOutputs([band]), 
  scale=image.projection().nominalScale()
)

# Print the value to test.
print(computation.first().get(band).getInfo())

### Serialize the expression graph

This will create an object that represents the Earth Engine expression graph (specifically, an [`Expression`](https://developers.google.com/earth-engine/reference/rest/v1alpha/Expression)).  In general, you should build these with one of the client APIs.

In [0]:
# Serialize the computation.
serialized = ee.serializer.encode(computation, for_cloud_api=True)

## Send the request

Make a `POST` request to the [`computeFeatures`](https://developers.google.com/earth-engine/reference/rest/v1alpha/projects.table/computeFeatures) endpoint.  Note that the request contains the [`Expression`](https://developers.google.com/earth-engine/reference/rest/v1alpha/Expression), which is the serialized computation.

In [0]:
import json

url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/table:computeFeatures'

response = session.post(
  url = url.format(PROJECT),
  data = json.dumps({'expression': serialized})
)

import json
pprint(json.loads(response.content))

The response contains the resultant `FeatureCollection` as GeoJSON, which can be consumed by other apps or processes.